# 공공데이터 API를 활용하여 데이터 수집하기

In [1]:
import json
import time
import xml.etree.ElementTree as ET
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import xmltodict       
import numpy as np

from tqdm import tqdm

In [175]:
# 예제

encoding = '비공개'
decoding = '비공개'


url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?_wadl&type=xml'
params ={'serviceKey' : encoding, 'LAWD_CD' : '11680', 'DEAL_YMD' : '202101' }

res = requests.get(url, params=params)
res

<Response [200]>

In [176]:
locCode = ["11110", "11140", "11170",
        "11200", "11215", "11230", 
        "11260", "11290", "11305", 
        "11320", "11350", "11380", 
        "11410", "11440", "11470", 
        "11500", "11530", "11545", 
        "11560", "11590", "11620", 
        "11650", "11680", "11710", 
        "11740"]

locCode_nm = ["종로구", "중구", "용산구", 
        "성동구", "광진구", "동대문구", 
        "중랑구", "성북구", "강북구", 
        "도봉구", "노원구", "은평구", 
        "서대문구", "마포구", "양천구", 
        "강서구", "구로구", "금천구", 
        "영등포구", "동작구", "관악구", 
        "서초구", "강남구", "송파구", 
        "강동구"]


# 2020~2022년간 구별 아파트 가격조회
year = [str("%02d" %(y))for y in range(2020,2022)]
month = [str("%02d" %(m))for m in range(1,13)]
yyyymm = ["%s%s" %(y,m) for y in year for m in month]

In [177]:
def get_items(get):
    root = ET.fromstring(get.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            data[tag] = text
        item_list.append(data)  
    return item_list

In [178]:
DataFrame = pd.DataFrame([])

for ym in tqdm(yyyymm):

    for code, name in zip(locCode, locCode_nm):

        time.sleep(0.5)
        url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?'
        url += 'serviceKey='+ encoding
        url += '&LAWD_CD='+ code
        url += '&DEAL_YMD='+ ym

        get = requests.get(url)
        data = json.loads(json.dumps(xmltodict.parse(get.text)))
        items = pd.DataFrame(data) 
        items["지역구"] = name
        DataFrame = pd.concat([DataFrame, items])

100%|██████████| 24/24 [07:10<00:00, 17.94s/it]


In [179]:
DataFrame

,response,지역구
body,"{'items': {'item': [{'거래금액': '100,000', '거래유형'...",종로구
header,"{'resultCode': '00', 'resultMsg': 'NORMAL SERV...",종로구
body,"{'items': {'item': [{'거래금액': '125,000', '거래유형'...",중구
header,"{'resultCode': '00', 'resultMsg': 'NORMAL SERV...",중구
body,"{'items': {'item': [{'거래금액': '91,000', '거래유형':...",용산구
...,...,...
header,"{'resultCode': '99', 'resultMsg': 'LIMITED NUM...",관악구
header,"{'resultCode': '99', 'resultMsg': 'LIMITED NUM...",서초구
header,"{'resultCode': '99', 'resultMsg': 'LIMITED NUM...",강남구
header,"{'resultCode': '99', 'resultMsg': 'LIMITED NUM...",송파구


LIMITED NUMBER OF SERVICE REQUESTS EXCEEDS ERROR - 일일사용량(트래픽) 초과로 부득이하게 API 활용은 어려운점 확인됨.

# 서울 열린데이터광장에서 CSV파일을 다운받아 진행

http://data.seoul.go.kr/dataList/datasetList.do

- 서울시 부동산 실거래가 정보
- 서울시 부동산 전월세가 정보

위 두가지 활용

## 데이터 불러오기

In [2]:
import pandas as pd

trade = 'C:/Users/JEON_SANGEON/codestates/project/toy/project_land/files/서울시 부동산 실거래가 정보.csv'
rent = 'C:/Users/JEON_SANGEON/codestates/project/toy/project_land/files/서울시 부동산 전월세가 정보.csv'

trade_df = pd.read_csv(trade, encoding='cp949')
rent_df = pd.read_csv(rent, encoding='cp949')
trade_df.head() # 20171230 ~ 20220609

C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\3806515845.py:6: DtypeWarning: Columns (6,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  trade_df = pd.read_csv(trade, encoding='cp949')
C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\3806515845.py:7: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  rent_df = pd.read_csv(rent, encoding='cp949')


,접수연도,자치구코드,자치구명,법정동명,법정동코드,지번구분명,본번,부번,건물명,계약일,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
0,2022,11545,금천구,시흥동,10300,일반,987.0,0.0,금천 쏠라 이지움,20220609,15000,17.58,30.33,6.0,NaN,NaN,2016.0,오피스텔,중개거래,서울 금천구
1,2022,11740,강동구,천호동,10900,일반,322.0,30.0,조은빌,20220609,24000,28.79,18.66,4.0,NaN,NaN,2014.0,연립다세대,중개거래,서울 강동구
2,2022,11470,양천구,신정동,10100,일반,979.0,11.0,롯데아트빌,20220609,22000,62.49,29.74,2.0,NaN,NaN,2002.0,연립다세대,직거래,NaN
3,2022,11740,강동구,길동,10500,일반,413.0,24.0,(413-24),20220609,13900,18.76,25.51,5.0,NaN,NaN,2016.0,오피스텔,중개거래,서울 강동구
4,2022,11740,강동구,성내동,10800,일반,244.0,5.0,한양립스,20220609,12800,12.33,0.00,5.0,NaN,NaN,2014.0,아파트,직거래,NaN


In [27]:
rent_df.head() # 20210204 ~ 20220609 까지만 있는것 확인됨. 추가 필요

,접수연도,자치구코드,자치구명,법정동명,지번구분명,본번,부번,층,계약일,전월세 구분,...,보증금(만원),임대료(만원),건물명,건축년도,건물용도,계약기간,신규갱신여부,계약갱신권사용여부,종전 보증금,종전 임대료
0,2022,11680,강남구,역삼동,NaN,NaN,NaN,NaN,20220609,월세,...,1000,100,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
1,2022,11680,강남구,논현동,NaN,NaN,NaN,NaN,20220609,전세,...,24000,0,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
2,2022,11215,광진구,중곡동,NaN,NaN,NaN,NaN,20220609,월세,...,1000,50,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
3,2022,11440,마포구,상암동,NaN,NaN,NaN,NaN,20220609,전세,...,12500,0,NaN,1997.0,단독다가구,NaN,NaN,NaN,NaN,NaN
4,2022,11530,구로구,개봉동,일반,497.0,0.0,8.0,20220609,월세,...,27891,43,센트레빌레우스1단지,2020.0,아파트,NaN,NaN,NaN,NaN,NaN


In [28]:
# 주요 컬럼 체크

trade_df.columns

Index(['접수연도', '자치구코드', '자치구명', '법정동명', '법정동코드', '지번구분명', '본번', '부번', '건물명',
       '계약일', '물건금액(만원)', '건물면적(㎡)', '토지면적(㎡)', '층', '권리구분', '취소일', '건축년도',
       '건물용도', '신고구분', '신고한 개업공인중개사 시군구명'],
      dtype='object')

In [29]:
rent_df.columns

Index(['접수연도', '자치구코드', '자치구명', '법정동명', '지번구분명', '본번', '부번', '층', '계약일',
       '전월세 구분', '임대면적(㎡)', '보증금(만원)', '임대료(만원)', '건물명', '건축년도', '건물용도',
       '계약기간', '신규갱신여부', '계약갱신권사용여부', '종전 보증금', '종전 임대료'],
      dtype='object')

전월세 자료는 txt로 나누어져 있어서 한번에 합치기 진행함.

In [3]:
# 여러 txt파일 한번에 가져와서 하나의 DataFrame으로 연결하기

import glob
# get data file names
path =r'C:/Users/JEON_SANGEON/codestates/project/toy/project_land/files'
filenames = glob.glob(path + "/*.txt")

li = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\1848818191.py:11: DtypeWarning: Columns (4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\1848818191.py:11: DtypeWarning: Columns (4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\1848818191.py:11: DtypeWarning: Columns (4,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\1848818191.py:11: DtypeWarning: Columns (4,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=None, header=0)
C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\18488181

In [31]:
# 주요 컬럼 확인

frame.columns

Index(['접수년도', '자치구코드', '자치구명', '법정동명', '지번구분', '본번', '부번', '층', '계약일',
       '전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물명', '건축년도', '건물용도', '계약기간',
       '신규계약구분', '갱신청구권사용', '종전보증금', '종전임대료'],
      dtype='object')

In [4]:
# 전월세 컬럼 불일치 확인 합치기전 컬럼 통일

frame.columns = ['접수연도', '자치구코드', '자치구명', '법정동명', '지번구분명', '본번', '부번', '층', '계약일',
       '전월세 구분', '임대면적(㎡)', '보증금(만원)', '임대료(만원)', '건물명', '건축년도', '건물용도',
       '계약기간', '신규갱신여부', '계약갱신권사용여부', '종전 보증금', '종전 임대료']

In [5]:
rent_df_ = pd.concat([rent_df,frame],axis=0)
rent_df_

,접수연도,자치구코드,자치구명,법정동명,지번구분명,본번,부번,층,계약일,전월세 구분,...,보증금(만원),임대료(만원),건물명,건축년도,건물용도,계약기간,신규갱신여부,계약갱신권사용여부,종전 보증금,종전 임대료
0,2022,11680,강남구,역삼동,NaN,NaN,NaN,NaN,20220609,월세,...,1000,100,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
1,2022,11680,강남구,논현동,NaN,NaN,NaN,NaN,20220609,전세,...,24000,0,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
2,2022,11215,광진구,중곡동,NaN,NaN,NaN,NaN,20220609,월세,...,1000,50,NaN,1996.0,단독다가구,NaN,NaN,NaN,NaN,NaN
3,2022,11440,마포구,상암동,NaN,NaN,NaN,NaN,20220609,전세,...,12500,0,NaN,1997.0,단독다가구,NaN,NaN,NaN,NaN,NaN
4,2022,11530,구로구,개봉동,일반,497.0,0.0,8.0,20220609,월세,...,27891,43,센트레빌레우스1단지,2020.0,아파트,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327492,2021,11305,강북구,미아동,NaN,NaN,NaN,NaN,20210731,월세,...,4000,120,NaN,1970.0,단독다가구,21.07~23.07,신규,NaN,NaN,NaN
2327493,2021,11260,중랑구,묵동,NaN,NaN,NaN,NaN,20210614,전세,...,17000,0,NaN,2019.0,단독다가구,21.07~23.07,신규,NaN,NaN,NaN
2327494,2021,11590,동작구,사당동,NaN,NaN,NaN,NaN,20210701,월세,...,5000,60,NaN,NaN,단독다가구,NaN,NaN,NaN,NaN,NaN
2327495,2021,11620,관악구,신림동,NaN,NaN,NaN,NaN,20211129,전세,...,26000,0,NaN,1987.0,단독다가구,NaN,NaN,NaN,NaN,NaN


In [6]:
# 전세 csv자료중 2021년도 중복값 제거

rent_df_ = rent_df_.drop_duplicates(['접수연도','자치구명','법정동명','본번','부번','층','계약일','전월세 구분','보증금(만원)','임대료(만원)','건물용도'], keep='first')
rent_df_.shape

(2507862, 21)

두 자료 통합

In [7]:
# 매매, 전세 자료 통합전 정리

# 매매 신규 컬럼
trade_df['계약구분'] = np.NaN
trade_df['계약구분'] = trade_df['계약구분'].fillna('매매')
trade_df['면적(㎡)'] = trade_df['건물면적(㎡)'] + trade_df['토지면적(㎡)']
# 컬럼명 변경
trade_df.rename(columns = {'물건금액(만원)' : '거래금액(만원)'}, inplace = True)

# 변경 컬럼 포함하여 사용할 내용만 남기기
trade_df_ = trade_df[['접수연도','자치구코드','자치구명','법정동명','층','계약일','계약구분',
'거래금액(만원)','건축년도','면적(㎡)','건물용도']]

trade_df_.head()

,접수연도,자치구코드,자치구명,법정동명,층,계약일,계약구분,거래금액(만원),건축년도,면적(㎡),건물용도
0,2022,11545,금천구,시흥동,6.0,20220609,매매,15000,2016.0,47.91,오피스텔
1,2022,11740,강동구,천호동,4.0,20220609,매매,24000,2014.0,47.45,연립다세대
2,2022,11470,양천구,신정동,2.0,20220609,매매,22000,2002.0,92.23,연립다세대
3,2022,11740,강동구,길동,5.0,20220609,매매,13900,2016.0,44.27,오피스텔
4,2022,11740,강동구,성내동,5.0,20220609,매매,12800,2014.0,12.33,아파트


In [8]:
# 전세 컬럼 수정

rent_df_.rename(columns = {'보증금(만원)' : '거래금액(만원)',
                        '전월세 구분' : '계약구분',
                        '임대면적(㎡)' : '면적(㎡)'
                        }, inplace = True)

rent_df_ = rent_df_[['접수연도','자치구코드','자치구명','법정동명','층','계약일','계약구분',
'거래금액(만원)','임대료(만원)','건축년도','면적(㎡)','건물용도']]

C:\Users\JEON_SANGEON\AppData\Local\Temp\ipykernel_25552\2637590734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_df_.rename(columns = {'보증금(만원)' : '거래금액(만원)',


In [9]:
df = pd.concat([trade_df_,rent_df_],axis=0)
df

,접수연도,자치구코드,자치구명,법정동명,층,계약일,계약구분,거래금액(만원),건축년도,면적(㎡),건물용도,임대료(만원)
0,2022,11545,금천구,시흥동,6.0,20220609,매매,15000,2016.0,47.91,오피스텔,NaN
1,2022,11740,강동구,천호동,4.0,20220609,매매,24000,2014.0,47.45,연립다세대,NaN
2,2022,11470,양천구,신정동,2.0,20220609,매매,22000,2002.0,92.23,연립다세대,NaN
3,2022,11740,강동구,길동,5.0,20220609,매매,13900,2016.0,44.27,오피스텔,NaN
4,2022,11740,강동구,성내동,5.0,20220609,매매,12800,2014.0,12.33,아파트,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2327478,2021,11320,도봉구,방학동,NaN,20210111,월세,5000,1989.0,243.00,단독다가구,15.0
2327482,2021,11620,관악구,봉천동,NaN,20210127,전세,20000,2018.0,245.00,단독다가구,0.0
2327484,2021,11590,동작구,사당동,NaN,20200503,전세,21300,1983.0,245.58,단독다가구,0.0
2327486,2021,11305,강북구,수유동,NaN,20200130,전세,12000,1986.0,246.48,단독다가구,0.0


In [10]:
# sql 적재시 NaN 값이 있을 경우 에러발생하므로 0으로 대체하기로함.

df = df.fillna(0)

최종자료 csv화 진행

In [53]:
# df.to_csv("C:\\Users\\JEON_SANGEON\\codestates\\project\\toy\\project_land\\files\\land.csv", 
#     sep=',' ,encoding="utf-8-sig")

In [11]:
df.to_csv("C:\\Users\\JEON_SANGEON\\codestates\\project\\toy\\project_land\\files\\land.csv", 
    sep=',')

이후 PostgreSQL로 DB적재 진행함.